In [2]:
import time
import numpy as np
import pandas as pd

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5

import signal
signal.signal(signal.SIGINT, signal.SIG_DFL)

import sys
from PyQt5.QtCore import QCoreApplication
import signal
signal.signal(signal.SIGINT, signal.SIG_DFL)

import sys
from PyQt5.QtCore import QCoreApplication

import time

### Read and plot 

In [3]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')   

In [5]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='x')
#use HTML tags to specify colour and size
# box = win.addViewBox()
# occupancyEstimate = pg.TextItem(text="Occupancy Estimate: 0", color=(255,255,255), 
#                                 anchor=(0.5,0.5), border='w', fill=(0, 0, 255, 100))
# box.addItem(occupancyEstimate)

occupancyEstimate = win.addLabel(text="Occupancy Estimate: 0")

In [6]:
def readMeasurements(file):
    #read in measurements from csv
    rawCentroidData = pd.read_csv(file, header=None)
    #find headers and frames within headers
    #each header has the structure X, Y, CentroidNumber
    #below each header is the frame data
    centroidFrames = list()
    headerFound = False
    for rowIndex in range(0, rawCentroidData.shape[0]):
        row = rawCentroidData.loc[rowIndex]
        if row[0] == 'FilteredRange': #header has been found
            if headerFound: 
                #actual data was found last frame and this frame actual data is found again
                #past frame ended so add to list
                centroidFrames.append(frame)
                frame = pd.DataFrame([])
            else:
                #header found
                headerFound = True
                frame = pd.DataFrame([])
            #data should be following
        elif headerFound:
            #actual data
            radius = np.float(row[0])
            theta = np.float(row[2])
            centroidNumber = int(row[4])
            if len(frame) == 0:
                frame = pd.DataFrame({'Range':radius, 'Theta':theta, 'CentroidNumber':centroidNumber}, index=range(1))
            else:
                data = pd.DataFrame({'Range':radius, 'Theta':theta, 'CentroidNumber':centroidNumber}, index=range(centroidNumber,centroidNumber+1))
                frame = pd.concat([frame,data])
    
    return centroidFrames

In [9]:
filePath = 'CentroidData.csv'

centroidFrames = readMeasurements(filePath)

#remove all information within the file
# open(filePath, "w").close()

#plot information
for frame in centroidFrames:
    #plot all required information
    if np.equal(frame.values, np.array([[0,0,0]])).all():
        continue
    #find x and y positions and occupancy number
    posX = np.multiply(frame['Range'], np.cos(frame['Theta']))
    posY = np.multiply(frame['Range'], np.sin(frame['Theta']))
    numberOfTargets = np.max(frame['CentroidNumber']) + 1
    
    s1.setData(posX, posY)
    message = "Occupancy Estimate: " + str(numberOfTargets)
    win.removeItem(occupancyEstimate)
    occupancyEstimate = win.addLabel(text=message)
    QtGui.QApplication.processEvents() 
    time.sleep(0.1)
    

    

KeyboardInterrupt: 